In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv("cleaned_data.csv")
df = df.sample(frac=1)
df['gender'].replace(["Male", "Female"], [0, 1], inplace=True)
df['disposition'].replace(["Discharge", "Admit"], [0,  1], inplace=True)
df['esi'] = df['esi'] - 1

In [ ]:
class_weights = dict(df['esi'].value_counts())
class_weights = {x: min(df.shape[0]/class_weights[x], 10) for x in class_weights}
print(class_weights)

In [ ]:
cc_cols = [x for x in df.columns if x[:3] == "cc_"]
med_cols = [x for x in df.columns if x[:5] == "meds_"]
with open("pmh_cols.txt") as f:
    pmh_cols = f.readlines()
pmh_cols = list(map(lambda x: x[:-1], pmh_cols))

x_cols = ["age", "gender"] + cc_cols + pmh_cols
y_cols = ["disposition"] + med_cols

esi_data = pd.get_dummies(df['esi'], dtype=int)

In [ ]:
val_split = 0.9

np_x_train = np.array(df[x_cols])[:int(val_split * df.shape[0]), :]
np_esi_train = np.array(esi_data)[:int(val_split * df.shape[0]), :]
np_y_train = np.array(df[y_cols])[:int(val_split * df.shape[0]), :]

np_x_val = np.array(df[x_cols])[int(val_split * df.shape[0]):, :]
np_esi_val = np.array(esi_data)[int(val_split * df.shape[0]):, :]
np_y_val = np.array(df[y_cols])[int(val_split * df.shape[0]):, :]

print(np_x_train.shape)
print(np_esi_train.shape)
print()
print(np_x_val.shape)
print(np_esi_val.shape)

In [ ]:
gen_model = tf.keras.Sequential([
    tf.keras.layers.Input(483),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(49, activation="sigmoid")
])

smooth = 1.
def dice_coef(y_true, y_pred):
    intersection = tf.keras.backend.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true) + tf.keras.backend.sum(y_pred) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

gen_model.compile(
    loss=dice_coef_loss,
    metrics=[
        "accuracy",
        dice_coef,
        tf.keras.metrics.AUC(curve="PR"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.01, decay_steps=5, decay_rate=0.99, staircase=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    min_lr=0.0005
)

gen_model.fit(
    x=np_x_train,
    y=np_y_train,
    epochs=30,
    validation_data=(np_x_val, np_y_val),
    batch_size=32,
    callbacks=[reduce_lr]
)

In [ ]:
gen_model.save("meds_admission.keras")

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.01, decay_steps=5, decay_rate=0.99, staircase=True
)

esi_model = tf.keras.Sequential([
    tf.keras.layers.Input(483),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(5, activation="softmax")
])

esi_model.compile(
    loss="categorical_crossentropy",
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
)

In [ ]:
esi_model.summary()

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    min_lr=0.0005
)

esi_model.fit(
    x=np_x_train,
    y=np_esi_train,
    epochs=30,
    validation_data=(np_x_val, np_esi_val),
    batch_size=32,
    callbacks=[reduce_lr]
)

In [ ]:
esi_model.save("65_model.keras")